In [117]:
path = "data/portfolio/portfolio1.xlsx"

In [118]:
import datetime
import numpy as np
import pandas as pd

df_portfolio = pd.read_excel(path)
df_portfolio.columns = [c.upper() for c in df_portfolio.columns]
df_portfolio

,DATE_PORTFOLIO,ID,DATE_OF_BIRTH,DATE_START_OF_COVER,SUM_INSURED,CURRENT_STATUS,SEX
0,2021-12-31,1,1976-04-23,2022-01-01,1000,ACTIVE,m
1,2021-12-31,2,1969-03-03,2022-01-01,100,ACTIVE,f
2,2021-12-31,3,1970-09-13,2022-01-01,5000,DIS1,f


In [119]:
# extract the relevant information
_portfolio_date_array = df_portfolio["DATE_PORTFOLIO"].unique()
assert len(_portfolio_date_array) == 1, "More than one portfolio date"
#portfolio_date = _portfolio_date_array[0].astype(datetime.datetime)
portfolio_date = pd.Timestamp(_portfolio_date_array[0]).to_pydatetime()
portfolio_date

datetime.datetime(2021, 12, 31, 0, 0)

In [120]:
type(portfolio_date) is datetime.datetime


True

In [121]:
def completed_months_to_date(date_col, the_date):
    """ Calculate completed months between the date col and `the_date`.
        The latter one can be a np.datetime series or a datetime.datetime. """
    class _dt:
        def __init__(self, year, month, day):
            self.year = year
            self.month = month
            self.day = day
                
    class _date_emul:
        def __init__(self, the_date):
            self.dt = _dt(the_date.year, the_date.month, the_date.day)
    
    if isinstance(the_date, datetime.datetime):
        the_date = _date_emul(the_date)
    
    # calculate the age in months at the portfolio_date
    age_in_months0 = \
        the_date.dt.year * 12 + the_date.dt.month \
        - date_col.dt.year * 12 - date_col.dt.month \
        - 1 \
        + (the_date.dt.day >= date_col.dt.day).astype(int)
    return age_in_months0.values

completed_months_to_date(df_portfolio["DATE_OF_BIRTH"], portfolio_date)

array([548, 633, 615], dtype=int64)

In [122]:
completed_months_to_date(df_portfolio["DATE_OF_BIRTH"], df_portfolio["DATE_START_OF_COVER"])

array([548, 633, 615], dtype=int64)

In [123]:
gender = df_portfolio["SEX"]
assert set(gender.unique()).issubset(set(("m", "f")))
gender = gender.values
gender

array(['m', 'f', 'f'], dtype=object)

In [127]:
from state import States
_unknown_status = set(df_portfolio["CURRENT_STATUS"]) - {s.name for s in States}
assert len(_unknown_status) == 0, "Unknow status " +  str(_unknown_status)

In [125]:
df_portfolio["CURRENT_STATUS"].map({st.name: st for st in States})

0    0
1    0
2    1
Name: CURRENT_STATUS, dtype: int64

In [126]:
df_portfolio["SUM_INSURED"].values.astype(np.float64)

array([1000.,  100., 5000.])